In [13]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("roster_reviews.sqlite.db")

course_descriptions_df = pd.read_sql_query("SELECT * FROM course_descriptions", conn)
courses_df = pd.read_sql_query("SELECT * FROM courses", conn)

# c = conn.cursor()
# res = c.execute("SELECT * FROM course_descriptions").fetchall()

conn.close()

from openai import AsyncOpenAI
import json


client = AsyncOpenAI(
    base_url="http://101.35.52.226:9090/v1",
    api_key='api-key',
    timeout=45,
)


async def chat(prompt, stream=False, temperature=0.0, n=1):
    response = await client.chat.completions.create(
        model="qwen-110b-chat",
        messages=[{"role": "user", "content": prompt}],
        stream=stream,
        max_tokens=512,
        temperature=temperature,
        n=n,
        stop=["<|endoftext|>", "<|im_end|>"],
    )
    if not stream:
        if n == 1:
            return response.choices[0].message.content.strip()
        return response.choices
    return response

# Extracting insights from courses & majors

same deal with professors, know what these descriptions and what not intends to tell you. What are the types of information that is contained in these scraped datas? 

- perceived prerequisites: what is actually needed to understand these. NOT the hard-written prerequisites list because that can be misleading and inaccurate, since some concepts are rarely used.
- contents: what is taught in this course? 
- outcomes: What can you now achieve from taking this class that you previously cannot? How does this class benefit you in terms of academic & career & personal growth goals? How does it fit into the prerequisites chain? 

with these categories, we can prompt engineer our LLM to extract that.

along with that job, i also want to batch in another task while we're at it:
- hard written prerequisites: output as a list of course codes string is fine
- distribution requirements: a list (multiselect) from a predefined set of distributions, rest is ignored.
- combined with: output a list of course codes from the natural language input.

In [47]:
EXTRACTION_COURSE_DESCRIPTION_PROMPT = """
### COURSE TITLE: {title}

### HARD_WRITTEN_PREREQUISITES
{prereqs}

### DESCRIPTION
{description}
IF NO DESCRIPTION IS AVAILABLE, OUTPUT EMPTY LIST. DO NOT MAKE ANY EDUCATED GUESS ON PURELY THE TITLE!

#### said outcomes. may not be provided in the course description.
{outcomes}

### OTHER INPUT INFORMATIONS
may or may not be provided.

combined_with: {combined_with}
distribution_requirements: {distributions}
dates: {dates}


### INSTRUCTION
- perceived_prerequisites: what is actually needed to understand these. NOT the hard-written prerequisites list because that can be misleading and inaccurate, since some concepts are rarely used. output a list of short and concise string capturing everything. short sentences suffice
- contents: what is taught in this course? output a list of short and concise string capturing everything. short sentences suffice
- outcomes: What can you now achieve from taking this class that you previously cannot? How does this class benefit you in terms of academic & career & personal growth goals? How does it fit into the prerequisites chain? output a list of short and concise string capturing everything. short sentences suffice
- hard_written_prerequisites: output as a list of course codes string is fine
- distributions: a list (multiselect) from a predefined set of distributions, rest is ignored.
- combined_with: output a list of course codes from the natural language input.

### OUTPUT REQUIREMENTS
DO NOT MAKE UP ANY INFORMATION. IF THE DESCRIPTION IS NOT AVIALBLE OR IS EMPTY OR SUCH, OUTPUT EMPTY STRINGS OR EMPTY LISTS FOR EVERY FIELD; DO NOT MAKE ANY EDUCATED GUESS ON PURELY THE TITLE!
If there is not enough information to extract these information, output "[]" as an empty JSON list.
JSON output must be compatible and must begin with {{ and end with }} so that python's json.loads can parse it.
No escape character is needed for ', and wrap string with double quotes.

### JSON DATA EXTRACTED (beginning with {{ and ending with }} WITHOUT ```json and ```):
"""

In [46]:
# course_descriptions_df.head()
courses_df.loc[courses_df["course_id"] == 127792]
course_descriptions_df.loc[course_descriptions_df["course_id"] == 127792]

,course_description_id,course_id,description,title,credits,prerequisites,when_offered,combined_with,distribution,outcomes,ts
1,5,127792,No description available.,Introduction to Digital Photography,3,None,None,,None,None,2024-07-03 03:45:00


In [44]:
title = "Introduction to Digital Photography"
course_id = course_descriptions_df.loc[
    course_descriptions_df["title"] == title, "course_id"
].values[0]
# course_descriptions_df.loc[course_descriptions_df["title"] == "Object-Oriented Programming and Data Structures"].values[0]

prompt = EXTRACTION_COURSE_DESCRIPTION_PROMPT.format(
    title=title,
    dates=courses_df.loc[courses_df["course_id"] == course_id, "dates"].values[0],
    prereqs=course_descriptions_df.loc[
        course_descriptions_df["course_id"] == course_id, "prerequisites"
    ].values[0],
    description=course_descriptions_df.loc[
        course_descriptions_df["course_id"] == course_id, "description"
    ].values[0],
    outcomes=course_descriptions_df.loc[
        course_descriptions_df["course_id"] == course_id, "outcomes"
    ].values[0],
    combined_with=course_descriptions_df.loc[
        course_descriptions_df["course_id"] == course_id, "combined_with"
    ].values[0],
    distributions=course_descriptions_df.loc[
        course_descriptions_df["course_id"] == course_id, "distribution"
    ].values[0],
)

response = await chat(prompt)
print(response)

# print(prompt, response)

{
  "perceived_prerequisites": [],
  "contents": [],
  "outcomes": [],
  "hard_written_prerequisites": [],
  "distributions": [],
  "combined_with": []
}


In [64]:
# the saved sqlite is not that clean, need to clean it up

# for example, for title Introduction to Asian American Studies, we have:
ids = """15108
72912
99534
127793
153603
179143
204619
15107""".split("\n")


for id in ids:
    print(id, courses_df.loc[courses_df["course_id"] == int(id)][['type', 'semester']].values[0:5])
    # print(id, course_descriptions_df.loc[course_descriptions_df["course_id"] == int(id)])

course_descriptions_df.loc[course_descriptions_df["course_id"] == 15108][['description', 'distribution']].values[0]
# courses_df.loc[courses_df["course_id"] == 15108]

15108 [['Project' 'SP24']]
72912 [['Lecture' 'SP22']]
99534 [['Lecture' 'SP21']]
127793 [['Lecture' 'SP20']]
153603 [['Lecture' 'SP19']]
179143 [['Lecture' 'SP18']]
204619 [['Lecture' 'SP17']]
15107 [['Lecture' 'SP24']]


array(['This interdisciplinary course offers an introduction to the study of Asian/Pacific Islanders in the U.S. This course will examine, through a range of disciplines (including history, literary studies,&nbsp;film/media, performance, anthropology, sociology), issues and methods that have emerged from Asian American Studies since its inception in the late 1960s, including the types of research questions and methods that the study of Asians & Pacific Islander peoples in the U.S. as well as politics and historical relations in the Asia/Pacific region have to offer. In this course, we will pay particular attention to the role of culture and its production in documenting histories, formulating critical practices, and galvanizing political efforts. Topics and themes include: war & empire; queer & feminist lives and histories; refugee, adoptees, transnational families, and other forms of kinship & belonging; anti-Asian violence; settler colonialism and postcolonial critique.',
       '(AL

In [65]:
import pandas as pd

as_details_df = pd.read_json("as-studies-details.json")
as_list_df = pd.read_json("as-studies-list.json")

In [86]:
EXTRACT_MAJOR_REQUIREMENTS_PROMPT = """
### MAJOR: {major}

### DESCRIPTION
{description}

### REQUIREMENTS
{requirements}

### OUTCOMES
{outcomes}

### OTHER INFORMATION for your reference (do not extract from these)
to better get an idea of where this studies would concentrate on, here are some more information that may or may not be helpful. Take it with a grain of salt when analyzing the data.
sample_classes: {sample_classes}
related_disciplines: {related_disciplines}
associated_interests: {associated_interests}

### INSTRUCTIONS
BELOW ARE THE FIELDS THAT NEED TO BE EXTRACTED. IF THE INFORMATION IS NOT AVAILABLE, OUTPUT EMPTY STRINGS OR EMPTY LISTS. DO NOT MAKE ANY EDUCATED GUESS ON PURELY THE TITLE!
- description: output a LIST of short and concise string capturing the DESCRIPTION section. This can but not limited to include what works are expected, what are covered, what are the goals of this study... 2-5 short sentences suffice
- outcomes: output a LIST of short and concise string capturing the OUTCOMES section. This can but not limited to include what can you achieve, where does this enable you to work in, is there anything that this major allows you to achieve, how does this study benefit you in terms of academic & career & personal growth goals... 2-5 short sentences suffice
- requirements: output a LIST of short and concise string capturing the REQUIREMENTS section. This can but not limited to include what are the requirements, what are the perceived prerequisites, what are the core courses... 2-5 short sentences suffice
using the above information, output a comprehensive JSON data that can be used for further analysis and recommendation.

### OUTPUT REQUIREMENTS
EXTRACT THE FIELDS AS SPECIFIED under INSTRUCTIONS.
DO NOT MAKE UP ANY INFORMATION. IF THE DESCRIPTION IS NOT AVIALBLE OR IS EMPTY OR SUCH, OUTPUT EMPTY STRINGS OR EMPTY LISTS FOR EVERY FIELD; DO NOT MAKE ANY EDUCATED GUESS ON PURELY THE TITLE!
If there is not enough information to extract these information, output "[]" as an empty JSON list.
JSON output must be compatible and must begin with {{ and end with }} so that python's json.loads can parse it.

### JSON DATA EXTRACTED (beginning with {{ and ending with }} WITHOUT ```json and ```):
"""

In [87]:
# print("\n".join(as_details_df['sidebar'][0]['associated_interests']))
# as_details_df.columns

sample_classes = as_details_df['sidebar'][0].get('sample_classes', [])
related_disciplines = as_details_df['sidebar'][0].get('related_disciplines', [])
associated_interests = as_details_df['sidebar'][0].get('associated_interests', [])

prompt = EXTRACT_MAJOR_REQUIREMENTS_PROMPT.format(
    major=as_details_df['title'][0],
    description=as_details_df['description'][0],
    requirements=as_details_df['requirements'][0],
    outcomes=as_details_df['outcomes'][0],
    sample_classes=sample_classes,
    related_disciplines=related_disciplines,
    associated_interests=associated_interests,
)
response = await chat(prompt)
print(response)

{
  "description": [
    "Explores the significance of Africa and its diaspora in global history and culture.",
    "Covers literature, history, philosophy, international relations, and the arts.",
    "Offers specialized focus through topics like history, literature, politics, and identity studies."
  ],
  "outcomes": [
    "Graduates pursue advanced degrees in diverse fields like African studies, history, and public policy.",
    "Enters careers in communications, hospitality, law, and arts management.",
    "Develops skills in critical thinking, cultural analysis, and social justice advocacy."
  ],
  "requirements": [
    "Requires 10 courses for the major, including an introduction to Africana studies.",
    "Three courses from African, African-American, Caribbean, or other diaspora studies.",
    "Four courses with a focus on two topics within the African diaspora at the 3000 level.",
    "Two seminar courses at the 4000 level.",
    "For a minor, five courses are required with si